# 2024 데이터 크리에이터 캠프

문제: 인공지능은 사람의 마음을 이해할수 있을까?

## Mission2. 패션 스타일 선호 여부 예측

라이브러리 불러오기

In [1]:
import os
import json
import csv
import pandas as pd
from collections import defaultdict, Counter

디렉토리 경로 지정

In [2]:
# 파일이 있는 디렉토리 경로
train_label_directory = '../dataset/training_label'
valid__label_directory = '../dataset/validation_label'
train_image_directory = '../dataset/training_image'
valid_image_directory = '../dataset/validation_image'

### Mission 2-1.  
“{W/T}_{이미지ID}_{시대별}_{스타일별}_{성별}_{설문ID}.json”이에  
기반하여 “설문ID” 수 기준으로 “성별 & 스타일” 통계치를 표 형식으로 기입한다.  

In [3]:
def count_images_by_gender_and_style(file_names, valid_image_ids):
    rows = []  # 결과를 저장할 리스트

    # 파일명 분석 및 카운트
    for filename in file_names:
        # 파일 확장자 체크
        if not filename.endswith('.json'):
            continue

        # .json 제거
        no_jpg_filename = filename.split('.')[0]
        parts = no_jpg_filename.split('_')

        # 성별, 스타일, 이미지 ID 추출
        gender = '여성' if parts[4] == 'W' else '남성'
        style = parts[3]  
        image_id = parts[1] 

        # 유효한 스타일과 이미지 ID일 때만 추가
        if style is not None and image_id is not None and image_id in valid_image_ids:
            rows.append({'성별': gender, '스타일': style, '이미지 ID': image_id})

    # DataFrame 생성
    df = pd.DataFrame(rows)

    # 중복된 행 제거
    df = df.drop_duplicates(subset=['성별', '스타일', '이미지 ID'])

    # 성별과 스타일별로 이미지 수 집계
    result = df.groupby(['성별', '스타일']).size().reset_index(name='이미지 수')
    result = result.sort_values(by='성별')
    
    return result

train 데이터

In [4]:
# 폴더 내의 파일 목록 가져오기
train_file_list_label = os.listdir(train_label_directory)
train_file_names_label = [filename for filename in train_file_list_label]

# 폴더 내의 파일 목록 가져오기
train_file_image_list = os.listdir(train_image_directory)
train_file_names = [filename for filename in train_file_image_list]

# 이미지 ID 추출
train_image_ID = list(set(name.split('_')[1] for name in train_file_names))

# traininig_image 폴더
result_train = count_images_by_gender_and_style(train_file_names_label, train_image_ID)
result_train.to_csv('mission2-1_train.csv', index=False)

In [5]:
result_train

,성별,스타일,이미지 수
0,남성,bold,312
1,남성,hiphop,353
2,남성,hippie,274
3,남성,ivy,256
4,남성,metrosexual,332
5,남성,mods,330
6,남성,normcore,754
7,남성,sportivecasual,687
21,여성,lounge,167
22,여성,military,36


validation 데이터

In [6]:
# 폴더 내의 파일 목록 가져오기
val_file_list_label = os.listdir(valid__label_directory)
val_file_names_label = [filename for filename in val_file_list_label]

val_file_list_image = os.listdir(valid_image_directory)
val_file_names = [filename for filename in val_file_list_image]

# 이미지 ID 추출
val_image_ID = list(set(name.split('_')[1] for name in val_file_names))

# validation_image 폴더
result_val = count_images_by_gender_and_style(val_file_names_label, val_image_ID)
result_val.to_csv('mission2-1_val.csv', index=False)

In [7]:
result_val

,성별,스타일,이미지 수
0,남성,bold,59
1,남성,hiphop,67
2,남성,hippie,82
3,남성,ivy,79
4,남성,metrosexual,58
5,남성,mods,81
6,남성,normcore,63
7,남성,sportivecasual,66
21,여성,lounge,12
22,여성,military,9


### Mission 2-2.  
2-1에서 구한 유효한 라벨링 데이터만 따로 분리하여 100명 응답자의 “스타일 선호 정보표”를 구한다.     
파일은 json 포맷으로 되어 있으며 json 필드 중, “응답자ID”는 “user>R_id”로 알 수 있고,  
“스타일 선호 여부”는 “item>survey>Q5”로 알 수 있다.    

In [8]:
# 결과를 저장할 딕셔너리
train_result = defaultdict(lambda: {'선호': [], '비선호': []})
valid_result = defaultdict(lambda: {'선호': [], '비선호': []})
response_count = Counter()

# 유효한 이미지 ID를 얻기 위한 함수
def count_images_by_gender_and_style(file_names, valid_image_ids):
    rows = []  # 결과를 저장할 리스트

    # 파일명 분석 및 카운트
    for filename in file_names:
        # 파일 확장자 체크
        if not filename.endswith('.json'):
            continue

        # .json 제거
        no_jpg_filename = filename.split('.')[0]
        parts = no_jpg_filename.split('_')

        # 성별, 스타일, 이미지 ID 추출
        gender = '여성' if parts[4] == 'W' else '남성'
        style = parts[3]  
        image_id = parts[1] 

        # 유효한 스타일과 이미지 ID일 때만 추가
        if style is not None and image_id is not None and image_id in valid_image_ids:
            rows.append({'성별': gender, '스타일': style, '이미지 ID': image_id})

    # DataFrame 생성
    df = pd.DataFrame(rows)

    # 중복된 행 제거
    df = df.drop_duplicates(subset=['성별', '스타일', '이미지 ID'])

    # 성별과 스타일별로 이미지 수 집계
    result = df.groupby(['성별', '스타일']).size().reset_index(name='이미지 수')
    result = result.sort_values(by='성별')
    
    return result

# 유효한 이미지 ID를 얻기 위한 파일 목록
train_files = os.listdir(train_label_directory)
valid_files = os.listdir(valid__label_directory)
train_image_files = os.listdir(train_image_directory)
valid_image_files = os.listdir(valid_image_directory)

# 이미지 파일에서 유효한 이미지 ID 추출
valid_image_ids = set([filename.split('_')[1] for filename in train_image_files + valid_image_files if filename.endswith('.jpg')])

# 유효한 데이터만 필터링
train_valid_images = count_images_by_gender_and_style(train_files + valid_files, valid_image_ids)

# 디렉토리 내의 모든 파일을 순회하며 데이터 추출
def process_directory(directory, result_dict, valid_image_ids):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                data = json.load(file)
                R_id = data['user']['R_id']
                imgName = data['item']['imgName']
                Q5 = data['item']['survey']['Q5']
                
                # 이미지 ID 추출
                image_id = imgName.split('_')[1]
                
                # 유효한 이미지 ID일 때만 처리
                if image_id in valid_image_ids:
                    # Q5 값을 선호도 값으로 변환
                    preference = "선호" if Q5 == 2 else "비선호"
                    
                    result_dict[R_id][preference].append(imgName)
                    response_count[R_id] += 1

# train과 valid 디렉토리 처리
process_directory(train_label_directory, train_result, valid_image_ids)
process_directory(valid__label_directory, valid_result, valid_image_ids)

# 결과를 CSV 파일로 저장
with open('mission2-2_result_all.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['응답자 ID', 'train 선호', 'train 비선호', 'valid 선호', 'valid 비선호']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for R_id in response_count:
        train_preference = ', '.join(train_result[R_id]['선호'])
        train_non_preference = ', '.join(train_result[R_id]['비선호'])
        valid_preference = ', '.join(valid_result[R_id]['선호'])
        valid_non_preference = ', '.join(valid_result[R_id]['비선호'])
        
        writer.writerow({
            '응답자 ID': R_id,
            'train 선호': train_preference,
            'train 비선호': train_non_preference,
            'valid 선호': valid_preference,
            'valid 비선호': valid_non_preference
        })

전체 응답자에 대한 선호여부

In [9]:
import pandas as pd

# 출력 옵션 설정
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# CSV 파일 읽기
result = pd.read_csv("mission2-2_result_all.csv")

result.tail()

,응답자 ID,train 선호,train 비선호,valid 선호,valid 비선호
4111,48227,NaN,NaN,NaN,W_41448_10_sportivecasual_W.jpg
4112,65051,NaN,NaN,NaN,W_44520_70_punk_W.jpg
4113,62013,NaN,NaN,NaN,W_52417_00_metrosexual_M.jpg
4114,66230,NaN,NaN,NaN,W_52521_50_ivy_M.jpg
4115,65680,NaN,NaN,W_53808_80_bold_M.jpg,NaN


In [10]:
len(result)

4116

응답자 100명에 대한 선호여부

In [11]:
# 상위 100명 데이터 추출
top_100 = result.head(100)

# 상위 100명 데이터를 새로운 CSV 파일로 저장
top_100.to_csv("mission2-2_result_top100.csv", index=False, encoding='utf-8-sig')

In [12]:
# CSV 파일 읽기
top_100_result = pd.read_csv("mission2-2_result_top100.csv")

top_100_result

,응답자 ID,train 선호,train 비선호,valid 선호,valid 비선호
0,52002,W_24111_70_hippie_M.jpg,"T_00004_90_hiphop_M.jpg, T_03007_10_sportivecasual_M.jpg, T_03118_19_normcore_M.jpg, W_02699_60_mods_M.jpg, W_23983_60_mods_M.jpg",NaN,NaN
1,66699,"T_00004_90_hiphop_M.jpg, T_01568_50_ivy_M.jpg, T_15877_10_sportivecasual_M.jpg, T_16259_10_sportivecasual_M.jpg, W_00901_60_mods_M.jpg","T_03643_00_metrosexual_M.jpg, T_06009_10_sportivecasual_M.jpg, W_15364_00_metrosexual_M.jpg, W_24031_60_mods_M.jpg",NaN,NaN
2,66797,"T_01259_10_sportivecasual_M.jpg, T_16092_10_sportivecasual_M.jpg, T_16332_10_sportivecasual_M.jpg, W_23958_60_mods_M.jpg, W_24020_60_mods_M.jpg","T_00004_90_hiphop_M.jpg, W_15467_70_hippie_M.jpg, W_24865_60_mods_M.jpg, W_49510_00_metrosexual_M.jpg","T_08486_10_sportivecasual_M.jpg, W_23958_60_mods_M.jpg",NaN
3,66684,"T_00047_19_normcore_M.jpg, T_03699_90_hiphop_M.jpg, W_02673_60_mods_M.jpg, W_32443_70_hippie_M.jpg","T_00007_19_normcore_M.jpg, W_51917_00_metrosexual_M.jpg",W_15341_60_mods_M.jpg,NaN
4,66817,"T_00012_19_normcore_M.jpg, T_04506_90_hiphop_M.jpg","T_03624_90_hiphop_M.jpg, T_04522_90_hiphop_M.jpg, W_16694_00_metrosexual_M.jpg",NaN,W_17135_00_metrosexual_M.jpg
...,...,...,...,...,...
95,67114,"T_01248_19_normcore_M.jpg, T_16051_60_mods_M.jpg","T_00410_50_ivy_M.jpg, T_03767_90_hiphop_M.jpg, T_15235_00_metrosexual_M.jpg",T_16067_60_mods_M.jpg,NaN
96,67139,"T_00410_50_ivy_M.jpg, T_00539_50_ivy_M.jpg, T_00545_50_ivy_M.jpg, T_06570_10_sportivecasual_M.jpg, T_06573_10_sportivecasual_M.jpg, T_06603_10_sportivecasual_M.jpg, T_06610_10_sportivecasual_M.jpg",NaN,NaN,NaN
97,67089,NaN,"T_00432_10_sportivecasual_M.jpg, T_03334_19_normcore_M.jpg",NaN,NaN
98,68396,"T_00432_10_sportivecasual_M.jpg, T_18080_19_normcore_M.jpg, T_18094_19_normcore_M.jpg, T_18095_19_normcore_M.jpg","T_18093_19_normcore_M.jpg, W_17455_00_metrosexual_M.jpg, W_17460_00_metrosexual_M.jpg, W_24537_70_hippie_M.jpg",NaN,W_24535_70_hippie_M.jpg


.